In [1]:
from numba import cuda,float32
import scipy.spatial as spatial
import numpy as np
#from rich import print
import matplotlib.pyplot as plt
from more_itertools import powerset
print(cuda.current_context().device)

<CUDA device 0 'b'NVIDIA GeForce GTX 1070 Ti''>


In [2]:
X = np.random.rand(4,3)
dimension = X.shape[1]
tri = spatial.Delaunay(X)
delaunay_complex = [ powerset(face) for face in tri.simplices ]
delaunay_complex = list(set([
    tuple(sorted(
        list(item) + [-1 for _ in range(len(item), dimension + 1)]
    )) for sublist in delaunay_complex for item in sublist
]) - {(-1,-1,-1,-1)})
filtration = np.zeros((len(delaunay_complex)))
filtration[:] = -1
delaunay_complex = np.asarray(delaunay_complex)
tri = np.asarray(tri.simplices)
print(tri)
print(delaunay_complex)
print(filtration)

[[2 1 3 0]]
[[-1 -1 -1  2]
 [-1 -1  0  3]
 [-1  1  2  3]
 [-1 -1  1  2]
 [ 0  1  2  3]
 [-1 -1 -1  0]
 [-1 -1 -1  3]
 [-1 -1  1  3]
 [-1 -1  0  1]
 [-1 -1  2  3]
 [-1  0  1  2]
 [-1 -1 -1  1]
 [-1  0  1  3]
 [-1 -1  0  2]
 [-1  0  2  3]]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


# Declare kernel function
https://numba.readthedocs.io/en/stable/cuda/kernels.html#kernel-declaration

When running a kernel, the kernel function’s code is executed by every thread once. It therefore has to know which thread it is in, in order to know which array element(s) it is responsible for (complex algorithms may define more complex responsibilities, but the underlying principle is the same).

![](https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png)

In [7]:
threadsperblock = 32
cayley_menger_shape = ( tri.shape[0], dimension + 2, dimension + 2)
subfaces_shape = (tri.shape[0], 2**(dimension+1) - (dimension + 2))
print('cayley_menger_shape', cayley_menger_shape)
blockspergrid = (tri.size + (threadsperblock - 1)) // threadsperblock
print('blockspergrid ', blockspergrid)

@cuda.jit
def alpha_complex(
    tri,             # Delaunay triangulation (d-simplices only). Every thread gets a d-simplex.
    delaunay_asc,    # Delaunay complex (abstract simplical complex). Threads use this to lookup where to store result.
    distance_matrix, # Pre-computed distance matrix.
    filtration       # Output array, the filtration values in the order of delaunay_asc.
):
    cayley_menger = cuda.shared.array(
        shape = cayley_menger_shape,
        dtype = float32
    )
    
    idx = cuda.grid(1)
    
    if idx < tri.shape[0]:
        # prepare Cayley Menger matrix
        cayley_menger[idx,:,1:] = 1
        cayley_menger[idx,1:,:] = 1
        cayley_menger[idx,0,0]  = 0
        

cayley_menger_shape (1, 5, 5)
blockspergrid  1


# Invoke kernel function
https://numba.readthedocs.io/en/stable/cuda/kernels.html#kernel-invocation

We need to compile the function first, which is done by invoking it for small valid input.
There is a limit to the number of threads per block, since all threads of a block are expected to reside on the same processor core and must share the limited memory resources of that core. On current GPUs, a thread block may contain up to 1024 threads.
https://docs.nvidia.com/cuda/cuda-c-programming-guide/#thread-hierarchy

![](https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/memory-hierarchy.png)

In [4]:
distance_matrix = spatial.distance_matrix(X,X)

In [5]:
alpha_complex[blockspergrid, threadsperblock](tri, delaunay_complex, distance_matrix, filtration)

/home/axean/anaconda3/envs/az1/lib/python3.9/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (38) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/axean/anaconda3/envs/az1/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:790: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


0
